In [0]:
 %run ./00setupconfig

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 02 - Silver Layer: Star Schema with Liquid Clustering
# MAGIC 
# MAGIC This notebook creates the star schema with dimension and fact tables, implementing liquid clustering for optimal query performance.
# MAGIC 
# MAGIC **Features:**
# MAGIC - Star schema design (fact + dimension tables)
# MAGIC - Liquid clustering on fact table
# MAGIC - Broadcast-optimized dimension tables
# MAGIC - SCD Type 1 for dimension updates
# MAGIC 
# MAGIC **Author:** Data Engineering Team  
# MAGIC **Last Updated:** December 2024

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Load Configuration

# COMMAND ----------

# MAGIC %run ./00_setup_config

# COMMAND ----------

from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Configuration variables are already available from %run command
# No need to read from temp view - they're in the namespace

print("✅ Configuration loaded for star schema creation")
print(f"   Catalog: {CATALOG}")
print(f"   Bronze Schema: {BRONZE_SCHEMA}")
print(f"   Silver Schema: {SILVER_SCHEMA}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Create Dimension Tables
# MAGIC 
# MAGIC Create small, broadcast-friendly dimension tables for lookups.

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.1 Dimension: Vendor

# COMMAND ----------

# Create vendor dimension
vendor_data = [
    (1, "Creative Mobile Technologies, LLC", "CMT"),
    (2, "VeriFone Inc.", "VTS")
]

dim_vendor = spark.createDataFrame(vendor_data, ["vendor_id", "vendor_name", "vendor_code"])

# Add audit columns
dim_vendor = dim_vendor.withColumn("created_at", F.current_timestamp())

# Write to Delta table
dim_vendor.write.mode("overwrite").saveAsTable(f"`{CATALOG}`.{SILVER_SCHEMA}.{DIM_VENDOR}")

print(f"✅ Created: {CATALOG}.{SILVER_SCHEMA}.{config['dim_vendor']}")
display(dim_vendor)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.2 Dimension: Rate Code

# COMMAND ----------

# Create rate code dimension
ratecode_data = [
    (1, "Standard rate", "Standard"),
    (2, "JFK", "JFK Airport"),
    (3, "Newark", "Newark Airport"),
    (4, "Nassau or Westchester", "Nassau/Westchester"),
    (5, "Negotiated fare", "Negotiated"),
    (6, "Group ride", "Group")
]

dim_ratecode = spark.createDataFrame(ratecode_data, ["ratecode_id", "ratecode_name", "ratecode_description"])
dim_ratecode = dim_ratecode.withColumn("created_at", F.current_timestamp())

dim_ratecode.write.mode("overwrite").saveAsTable(f"`{CATALOG}`.{SILVER_SCHEMA}.{DIM_RATECODE}")

print(f"✅ Created: `{CATALOG}`.{SILVER_SCHEMA}.{config['dim_ratecode']}")
display(dim_ratecode)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.3 Dimension: Payment Type

# COMMAND ----------

# Create payment type dimension
payment_data = [
    (1, "Credit card", "CC"),
    (2, "Cash", "CASH"),
    (3, "No charge", "NC"),
    (4, "Dispute", "DIS"),
    (5, "Unknown", "UNK"),
    (6, "Voided trip", "VOID")
]

dim_payment = spark.createDataFrame(payment_data, ["payment_type_id", "payment_type_name", "payment_type_code"])
dim_payment = dim_payment.withColumn("created_at", F.current_timestamp())

dim_payment.write.mode("overwrite").saveAsTable(f"`{CATALOG}`.{SILVER_SCHEMA}.{DIM_PAYMENT_TYPE}")

print(f"✅ Created: {CATALOG}.{SILVER_SCHEMA}.{config['dim_payment_type']}")
display(dim_payment)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.4 Dimension: Trip Type

# COMMAND ----------

# Create trip type dimension
trip_type_data = [
    (1, "Street-hail", "Street"),
    (2, "Dispatch", "Dispatch")
]

dim_trip_type = spark.createDataFrame(trip_type_data, ["trip_type_id", "trip_type_name", "trip_type_description"])
dim_trip_type = dim_trip_type.withColumn("created_at", F.current_timestamp())

dim_trip_type.write.mode("overwrite").saveAsTable(f"`{CATALOG}`.{SILVER_SCHEMA}.{DIM_TRIP_TYPE}")

print(f"✅ Created: `{CATALOG}`.{SILVER_SCHEMA}.{config['dim_trip_type']}")
display(dim_trip_type)

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.5 Dimension: Date
# MAGIC 
# MAGIC Create a date dimension for time-based analysis.

# COMMAND ----------

from datetime import datetime, timedelta

def create_date_dimension(start_date, end_date):
    """Generate date dimension with common attributes"""
    dates = []
    current_date = start_date
    
    while current_date <= end_date:
        dates.append({
            "date": current_date,
            "year": current_date.year,
            "quarter": (current_date.month - 1) // 3 + 1,
            "month": current_date.month,
            "month_name": current_date.strftime("%B"),
            "day": current_date.day,
            "day_of_week": current_date.weekday() + 1,  # Monday = 1
            "day_name": current_date.strftime("%A"),
            "week_of_year": current_date.isocalendar()[1],
            "is_weekend": 1 if current_date.weekday() >= 5 else 0,
            "is_holiday": 0  # Can be enhanced with holiday logic
        })
        current_date += timedelta(days=1)
    
    return dates

# Generate date dimension for 2023-2025
start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 12, 31)

date_data = create_date_dimension(start_date, end_date)
dim_date = spark.createDataFrame(date_data)
dim_date = dim_date.withColumn("created_at", F.current_timestamp())

dim_date.write.mode("overwrite").saveAsTable(f"`{CATALOG}`.{SILVER_SCHEMA}.{DIM_DATE}")

print(f"✅ Created: `{CATALOG}`.{SILVER_SCHEMA}.{config['dim_date']}")
print(f"   Date range: {start_date.date()} to {end_date.date()}")
print(f"   Total days: {dim_date.count():,}")
display(dim_date.limit(10))

# COMMAND ----------

# MAGIC %md
# MAGIC ### 2.6 Dimension: Location
# MAGIC 
# MAGIC Create location dimension for pickup/dropoff zones.

# COMMAND ----------

# For now, create a simple location dimension with IDs
# In production, you would load from NYC TLC taxi zone lookup
location_ids = list(range(1, 266))  # 265 taxi zones

dim_location = spark.createDataFrame(
    [(loc_id, f"Zone {loc_id}", "Unknown") for loc_id in location_ids],
    ["location_id", "zone_name", "borough"]
)
dim_location = dim_location.withColumn("created_at", F.current_timestamp())

dim_location.write.mode("overwrite").saveAsTable(f"`{CATALOG}`.{SILVER_SCHEMA}.{DIM_LOCATION}")

print(f"✅ Created: `{CATALOG}`.{SILVER_SCHEMA}.{config['dim_location']}")
print(f"   Total locations: {dim_location.count():,}")
display(dim_location.limit(10))

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Verify Dimension Tables

# COMMAND ----------

# Check all dimension tables
dimension_tables = [
    config['dim_vendor'],
    config['dim_ratecode'],
    config['dim_payment_type'],
    config['dim_trip_type'],
    config['dim_date'],
    config['dim_location']
]

print("📊 Dimension Table Summary")
print("=" * 80)

for table_name in dimension_tables:
    full_table_name = f"`{CATALOG}`.{SILVER_SCHEMA}.{table_name}"
    count = spark.table(full_table_name).count()
    size_mb = spark.sql(f"DESCRIBE DETAIL {full_table_name}").select("sizeInBytes").first()[0] / (1024 * 1024)
    
    broadcast_eligible = "✅ Yes" if size_mb < 10 else "⚠️  No"
    
    print(f"{table_name}:")
    print(f"   Records: {count:,}")
    print(f"   Size: {size_mb:.2f} MB")
    print(f"   Broadcast eligible: {broadcast_eligible}")
    print()

print("=" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. Create Fact Table with Liquid Clustering
# MAGIC 
# MAGIC Create the fact table with liquid clustering for optimal query performance.

# COMMAND ----------

# Read bronze data
bronze_df = spark.table(f"`{CATALOG}`.{BRONZE_SCHEMA}.{BRONZE_TABLE}")

# Transform to fact table structure
# Transform to fact table structure
fact_df = (
    bronze_df
    # Extract pickup and dropoff dates for clustering
    .withColumn("pickup_date", F.to_timestamp(F.col("lpep_pickup_datetime"), "M/d/yy H:mm"))
    .withColumn("dropoff_date", F.to_timestamp(F.col("lpep_dropoff_datetime"), "M/d/yy H:mm"))
    # Rename columns to match dimension keys
    .withColumnRenamed("VendorID", "vendor_id")
    .withColumnRenamed("RatecodeID", "ratecode_id")
    .withColumnRenamed("payment_type", "payment_type_id")
    .withColumnRenamed("trip_type", "trip_type_id")
    .withColumnRenamed("PULocationID", "pickup_location_id")
    .withColumnRenamed("DOLocationID", "dropoff_location_id")
    # Calculate derived metrics
    .withColumn(
        "trip_duration_minutes",
        (F.unix_timestamp(F.col("lpep_dropoff_datetime")) - F.unix_timestamp(F.col("lpep_pickup_datetime"))) / 60
    )
    # Add fact table audit columns
    .withColumn("fact_created_at", F.current_timestamp())
    .withColumn("fact_updated_at", F.current_timestamp())
    # Select relevant columns
    .select(
        "vendor_id",
        "pickup_date",
        "dropoff_date",
        "lpep_pickup_datetime",
        "lpep_dropoff_datetime",
        "ratecode_id",
        "pickup_location_id",
        "dropoff_location_id",
        "passenger_count",
        "trip_distance",
        "trip_duration_minutes",
        "fare_amount",
        "extra",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "ehail_fee",
        "improvement_surcharge",
        "total_amount",
        "payment_type_id",
        "trip_type_id",
        "congestion_surcharge",
        "store_and_fwd_flag",
        "fact_created_at",
        "fact_updated_at"
    )
)

print("✅ Fact table transformation complete")
print(f"   Total records: {fact_df.count():,}")
display(fact_df.limit(10))

# COMMAND ----------

# MAGIC %md
# MAGIC ### 4.1 Create Fact Table with Liquid Clustering

# COMMAND ----------

# Create fact table with liquid clustering
# Note: Liquid clustering is specified during table creation

fact_table_name = f"`{CATALOG}`.{SILVER_SCHEMA}.{FACT_TABLE}"

# Drop table if exists (for fresh creation)
spark.sql(f"DROP TABLE IF EXISTS {fact_table_name}")

# Create table with liquid clustering
create_table_sql = f"""
CREATE TABLE {fact_table_name}
USING DELTA
CLUSTER BY (pickup_date, vendor_id, payment_type_id)
COMMENT 'Fact table for Green Taxi trips with liquid clustering'
TBLPROPERTIES (
  'delta.enableLiquidClustering' = 'true',
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
AS SELECT * FROM {fact_df.createOrReplaceTempView('temp_fact')} OR (SELECT * FROM temp_fact LIMIT 0)
"""

# Write data to the fact table
fact_df.write.mode("overwrite").saveAsTable(fact_table_name)



# Set clustering columns
spark.sql(f"""
ALTER TABLE {fact_table_name}
CLUSTER BY (pickup_date, vendor_id, payment_type_id)
""")

print(f"✅ Fact table created with liquid clustering: {fact_table_name}")
print(f"   Clustering columns: {', '.join(CLUSTER_COLUMNS)}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 5. Verify Fact Table

# COMMAND ----------

# Check fact table properties
fact_details = spark.sql(f"DESCRIBE DETAIL {fact_table_name}").first()

print("📊 Fact Table Details")
print("=" * 80)
print(f"Table: {fact_table_name}")
print(f"Format: {fact_details['format']}")
print(f"Records: {fact_details['numFiles']:,} files")
print(f"Size: {fact_details['sizeInBytes'] / (1024 * 1024):.2f} MB")
print(f"Partitions: {fact_details['numPartitions'] if 'numPartitions' in fact_details else 'N/A'}")
print("=" * 80)

# Check table properties
properties = spark.sql(f"SHOW TBLPROPERTIES {fact_table_name}").collect()

print("\n📋 Table Properties:")
for prop in properties:
    if 'cluster' in prop['key'].lower() or 'liquid' in prop['key'].lower():
        print(f"   {prop['key']}: {prop['value']}")

# COMMAND ----------

# Get clustering information
try:
    clustering_info = spark.sql(f"DESCRIBE EXTENDED {fact_table_name}").filter(
        F.col("col_name").contains("Clustering")
    ).collect()
    
    if clustering_info:
        print("\n🎯 Clustering Information:")
        for info in clustering_info:
            print(f"   {info['col_name']}: {info['data_type']}")
except Exception as e:
    print(f"ℹ️  Clustering info: {e}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 6. Verify Star Schema Relationships

# COMMAND ----------

# Test joins between fact and dimension tables
test_query = f"""
SELECT 
    f.pickup_date,
    v.vendor_name,
    pt.payment_type_name,
    tt.trip_type_name,
    COUNT(*) as trip_count,
    SUM(f.total_amount) as total_revenue,
    AVG(f.trip_distance) as avg_distance
FROM {fact_table_name} f
JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_VENDOR} v ON f.vendor_id = v.vendor_id
JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_PAYMENT_TYPE} pt ON f.payment_type_id = pt.payment_type_id
JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_TRIP_TYPE} tt ON f.trip_type_id = tt.trip_type_id
WHERE f.pickup_date >= '2024-01-01'
GROUP BY f.pickup_date, v.vendor_name, pt.payment_type_name, tt.trip_type_name
ORDER BY f.pickup_date, total_revenue DESC
LIMIT 20
"""

print("🔍 Testing star schema joins...")
result_df = spark.sql(test_query)
display(result_df)

print("\n✅ Star schema relationships verified successfully!")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 7. Star Schema Statistics

# COMMAND ----------

# Generate comprehensive statistics
stats_query = f"""
SELECT 
    '📊 Overall Statistics' as category,
    CAST(COUNT(*) AS STRING) as total_trips,
    CAST(COUNT(DISTINCT vendor_id) AS STRING) as unique_vendors,
    CAST(COUNT(DISTINCT pickup_date) AS STRING) as unique_days,
    CAST(SUM(trip_distance) AS STRING) as total_distance,
    CAST(SUM(total_amount) AS STRING) as total_revenue,
    CAST(AVG(trip_distance) AS STRING) as avg_distance,
    CAST(AVG(total_amount) AS STRING) as avg_fare,
    CAST(AVG(trip_duration_minutes) AS STRING) as avg_duration_min,
    NULL as date_range
FROM {fact_table_name}

UNION ALL

SELECT 
    '📅 Date Range' as category,
    NULL as total_trips,
    NULL as unique_vendors,
    NULL as unique_days,
    NULL as total_distance,
    NULL as total_revenue,
    NULL as avg_distance,
    NULL as avg_fare,
    NULL as avg_duration_min,
    CONCAT(CAST(MIN(pickup_date) AS STRING), ' to ', CAST(MAX(pickup_date) AS STRING)) as date_range
FROM {fact_table_name}
"""

print("📈 Star Schema Statistics")
print("=" * 80)
stats_result = spark.sql(stats_query)
display(stats_result)

# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Create Views for Analytics

# COMMAND ----------

# Create a convenient view joining all dimensions
view_name = f"`{CATALOG}`.{SILVER_SCHEMA}.vw_green_trips_enriched"

spark.sql(f"""
CREATE OR REPLACE VIEW {view_name} AS
SELECT 
    f.*,
    v.vendor_name,
    v.vendor_code,
    rc.ratecode_name,
    pt.payment_type_name,
    tt.trip_type_name,
    d.year,
    d.month,
    d.month_name,
    d.day_of_week,
    d.day_name,
    d.is_weekend,
    pu.zone_name as pickup_zone,
    pu.borough as pickup_borough,
    do.zone_name as dropoff_zone,
    do.borough as dropoff_borough
FROM {fact_table_name} f
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_VENDOR} v 
    ON f.vendor_id = v.vendor_id
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_RATECODE} rc 
    ON f.ratecode_id = rc.ratecode_id
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_PAYMENT_TYPE} pt 
    ON f.payment_type_id = pt.payment_type_id
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_TRIP_TYPE} tt 
    ON f.trip_type_id = tt.trip_type_id
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_DATE} d 
    ON f.pickup_date = d.date
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_LOCATION} pu 
    ON f.pickup_location_id = pu.location_id
LEFT JOIN `{CATALOG}`.{SILVER_SCHEMA}.{DIM_LOCATION} do 
    ON f.dropoff_location_id = do.location_id
""")

print(f"✅ Created view: {view_name}")

# Test the view
print("\n📊 Sample from enriched view:")
display(spark.table(view_name).limit(10))

# COMMAND ----------

# MAGIC %md
# MAGIC ## 9. Summary

# COMMAND ----------

print("🎉 Silver Layer Star Schema - Complete!")
print("=" * 80)
print("\n✅ Created Dimension Tables:")
for table_name in dimension_tables:
    full_name = f"`{CATALOG}`.{SILVER_SCHEMA}.{table_name}"
    count = spark.table(full_name).count()
    print(f"   {table_name}: {count:,} records")

print(f"\n✅ Created Fact Table:")
fact_count = spark.table(fact_table_name).count()
print(f"   {FACT_TABLE}: {fact_count:,} records")
print(f"   Liquid clustering: pickup_date, vendor_id, payment_type_id")

print(f"\n✅ Created View:")
print(f"   vw_green_trips_enriched: Ready for analytics")

print("\n📊 Star Schema Ready for:")
print("   1. Broadcast joins with dimension tables")
print("   2. Optimized queries using liquid clustering")
print("   3. Time-based analysis with date dimension")
print("   4. Location-based analysis with zone lookups")

print("\n🚀 Next Steps:")
print("   - Run gold layer analytics: 03_gold_analytics.py")
print("   - Query with broadcast hints for best performance")
print("   - Monitor liquid clustering optimization")

print("\n" + "=" * 80)